# Cr₂ Potential Energy Surface Analysis

**Companion code for:** Kohn-Sham density encoding rescues coupled cluster theory for strongly correlated molecules\
**Journal:** *Nat. Comm.*\
**Authors:** Abdulrahman Y. Zamani, Barbaro Zulueta, Andrew M. Ricciuti, John A. Keith, Kevin Carter-Fenk \
**DOI:** TBA [DOI]

---

## Overview

This notebook reproduces the Cr₂ potential energy surface (PES) analysis presented in the manuscript. We demonstrate that CCSD(T) calculations using DFT references (CCSD(T)@DFT) can achieve chemical accuracy for the notoriously difficult Cr₂ dimer—a system traditionally requiring expensive multireference methods.

### What This Notebook Does

1. **Parses** ORCA output files from CCSD(T) calculations with various reference methods
2. **Computes** binding energies relative to atomic asymptotes
3. **Generates** figures comparing methods against experiment
4. **Exports** diagnostic data (T1, ⟨S²⟩) for further analysis at the QZ level of theory

### Computational Methods Compared

| Label | Reference | Description |
|-------|-----------|-------------|
| UCC-HF | UHF | Standard UCCSD(T) with Hartree-Fock reference |
| UCC-SVWN5 | USVWN5 | UCCSD(T) with LDA reference |
| UCC-PBE | UPBE | UCCSD(T) with PBE GGA reference |
| UCC-PW91 | UPW91 | UCCSD(T) with PW91 GGA reference |
| UCC-R²SCAN | UR²SCAN | UCCSD(T) with R²SCAN meta-GGA reference |
| UCC-PBE0 | UPBE0 | UCCSD(T) with PBE0 hybrid reference |

---

## 1. Setup and Requirements

### Dependencies

```
numpy>=1.20
matplotlib>=3.5
plotly>=5.0
```

Install with: `pip install numpy matplotlib plotly`

### Directory Structure

```
.
├── cr2_pes/                    # PES scan data
│   ├── ccsdt_hf/               # HF reference
│   │   ├── Cr-Cr001.out
│   │   ├── Cr-Cr002.out
│   │   └── ...
│   ├── ccsdt_pbe/              # PBE reference
│   ├── ccsdt_pw91/             # PW91 reference
│   ├── ccsdt_r2scan/           # R²SCAN reference
│   ├── ccsdt_svwn5/            # SVWN5 reference
│   ├── ccsdt_pbe0/             # PBE0 reference
│   ├── exp.txt                 # Experimental PES (Larsson 2022)
│   └── bte.txt                 # Best theoretical estimate (Larsson 2022)
│
└── species/                    # Atomic/molecular reference calculations
    └── Cr/                     # Cr atom
        ├── Cr_ccsdt_hf_cbs_x2c.out
        ├── Cr_ccsdt_pbe_cbs_x2c.out
        ├── Cr_ccsdt_pw91_cbs_x2c.out
        ├── Cr_ccsdt_r2scan_cbs_x2c.out
        ├── Cr_ccsdt_svwn5_cbs_x2c.out
        └── Cr_ccsdt_pbe0_cbs_x2c.out
```

In [10]:
from pathlib import Path
from dataclasses import dataclass, field
from typing import Optional
import warnings

import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

print(f"NumPy version: {np.__version__}")
print("Setup complete!")

NumPy version: 2.0.2
Setup complete!


---

## 2. Configuration

In [11]:
# =============================================================================
# USER CONFIGURATION
# =============================================================================

# System specification
SPECIES = 'Cr-Cr'
ATOMS = ['Cr', 'Cr']

# Computational methods to analyze
METHODS = [
    'ccsdt_hf',       # HF reference (standard CCSD(T))
    'ccsdt_svwn5',    # SVWN5 (LDA) reference
    'ccsdt_pbe',      # PBE reference
    'ccsdt_pw91',     # PW91 reference  
    'ccsdt_r2scan',   # R²SCAN reference
    'ccsdt_pbe0',     # PBE0 reference
]

# Directory paths
BASE_DIR = Path('cr2_pes')
SPECIES_DIR = Path('species')
OUTPUT_DIR = Path('cr2_multi_diag')

# Method name mapping for atomic calculation files
# Maps PES folder names to atomic file method names
METHOD_FILE_MAP = {
    'ccsdt_hf': 'ccsdt_hf',        # HF reference
    'ccsdt_svwn5': 'ccsdt_svwn5',
    'ccsdt_pbe': 'ccsdt_pbe',
    'ccsdt_pw91': 'ccsdt_pw91',
    'ccsdt_r2scan': 'ccsdt_r2scan',
    'ccsdt_pbe0': 'ccsdt_pbe0',
}

# Unit conversion factors
HARTREE_TO_KCAL = 627.5096
HARTREE_TO_EV = 27.211
KCAL_TO_EV = 23.06

# Verify configuration
print(f"System: {SPECIES}")
print(f"Methods to analyze: {len(METHODS)}")
print(f"Base directory: {BASE_DIR}")

System: Cr-Cr
Methods to analyze: 6
Base directory: cr2_pes


---

## 3. Data Structures

In [12]:
@dataclass
class MoleculeData:
    """
    Data extracted from an ORCA molecular calculation.
    
    Attributes
    ----------
    geometries : np.ndarray
        Cartesian coordinates (N_atoms × 3) in Ångströms
    atom_types : list
        Element symbols for each atom
    energy : float
        CBS-extrapolated CCSD(T) energy in Hartrees
    hf_scf_energies : np.ndarray
        HF/DFT SCF energies at each basis set level
    corr_energies : np.ndarray
        Correlation energies at each basis set level
    dft_energies : np.ndarray
        Reference DFT energies (for CCSD(T)@DFT calculations) 
        at QZ level of theory
    s2 : float
        ⟨S²⟩ expectation value (spin contamination diagnostic)
        at QZ level of theory
    t1 : float
        T1 diagnostic (single-reference reliability metric)
        at QZ level of theory
    """
    geometries: np.ndarray
    atom_types: list
    energy: float
    hf_scf_energies: np.ndarray
    corr_energies: np.ndarray
    dft_energies: np.ndarray
    s2: float
    t1: float


@dataclass
class AtomData:
    """Data extracted from an ORCA atomic calculation."""
    energy: float
    hf_scf_energies: np.ndarray
    corr_energies: np.ndarray
    dft_energies: np.ndarray


@dataclass
class MethodResults:
    """Collected results for a single computational method across the PES."""
    energies: list = field(default_factory=list)
    distances: list = field(default_factory=list)
    scf_energies: list = field(default_factory=list)
    t1_diagnostics: list = field(default_factory=list)
    s2_values: list = field(default_factory=list)
    bde: Optional[np.ndarray] = None
    scf_bde: Optional[np.ndarray] = None

---

## 4. ORCA Output Parsing Functions

In [13]:
def read_orca_molecule(path: Path) -> MoleculeData:
    """
    Parse ORCA output file for a molecular CCSD(T) calculation.
    
    Extracts geometry, CBS-extrapolated energies, and diagnostics (S², T1).
    
    Parameters
    ----------
    path : Path
        Path to the ORCA .out file
        
    Returns
    -------
    MoleculeData
        Parsed calculation data
    """
    geometries, atom_types = [], []
    dft_energies, hf_scf_energies, corr_energies = [], [], []
    energy, s2, t1 = None, None, None
    
    with open(path) as f:
        lines = f.readlines()[3:]
    
    geom_parsed, diag_parsed = False, False
    
    for i, line in enumerate(lines):
        # Parse geometry after first CBS extrapolation marker
        if 'Extrapolated Energy 1 Basis=  def2-TZVPP' in line and not geom_parsed:
            for j, subline in enumerate(lines[i+1:], start=i+1):
                if 'CARTESIAN COORDINATES (ANGSTROEM)' in subline:
                    for coord_line in lines[j+2:]:
                        if coord_line.strip() == '':
                            geom_parsed = True
                            break
                        parts = coord_line.split()
                        atom_types.append(parts[0])
                        geometries.append([float(x) for x in parts[1:4]])
                    break
        
        # Parse S² and T1 after second CBS extrapolation marker
        elif 'Extrapolated Energy 2 Basis=  def2-QZVPP' in line and not diag_parsed:
            for subline in lines[i+1:]:
                if 'Expectation value of <S**2>' in subline:
                    s2 = float(subline.split()[-1])
                elif 'T1 diagnostic' in subline:
                    t1 = float(subline.split()[-1])
                    diag_parsed = True
                    break
        
        # Collect DFT reference energies
        elif line.startswith('Total Energy       : '):
            energy_str = line[21:53].split()[0].replace('Eh', '')
            dft_energies.append(float(energy_str))
        
        # Parse CBS-extrapolated final energies
        elif 'Extrapolation of energy' in line:
            for subline in lines[i+1:]:
                if 'FINAL SINGLE POINT ENERGY' in subline:
                    energy = float(subline.split()[-1])
                    break
                elif 'SCF energy with basis' in subline:
                    hf_scf_energies.append(float(subline.split()[-1]))
                elif 'Correlation energy with basis' in subline:
                    corr_energies.append(float(subline.split()[-1]))
    
    return MoleculeData(
        geometries=np.array(geometries, dtype=np.float64),
        atom_types=atom_types,
        energy=energy,
        hf_scf_energies=np.array(hf_scf_energies),
        corr_energies=np.array(corr_energies),
        dft_energies=np.array(dft_energies[1:]) if len(dft_energies) > 1 else np.array(dft_energies),
        s2=s2,
        t1=t1
    )

In [14]:
def read_orca_atom(path: Path) -> AtomData:
    """
    Parse ORCA output file for an atomic CCSD(T) calculation.
    
    Parameters
    ----------
    path : Path
        Path to the ORCA .out file
        
    Returns
    -------
    AtomData
        Parsed atomic calculation data
    """
    dft_energies, hf_scf_energies, corr_energies = [], [], []
    energy = None
    
    with open(path) as f:
        lines = f.readlines()[3:]
    
    for i, line in enumerate(lines):
        if 'Extrapolation of energy' in line:
            for subline in lines[i+1:]:
                if 'FINAL SINGLE POINT ENERGY' in subline:
                    energy = float(subline.split()[-1])
                    break
                elif 'SCF energy with basis' in subline:
                    hf_scf_energies.append(float(subline.split()[-1]))
                elif 'Correlation energy with basis' in subline:
                    corr_energies.append(float(subline.split()[-1]))
        elif line.startswith('Total Energy       : '):
            energy_str = line[21:53].split()[0].replace('Eh', '')
            dft_energies.append(float(energy_str))
    
    return AtomData(
        energy=energy,
        hf_scf_energies=np.array(hf_scf_energies),
        corr_energies=np.array(corr_energies),
        dft_energies=np.array(dft_energies[1:]) if len(dft_energies) > 1 else np.array(dft_energies)
    )

In [15]:
def read_reference_data(filepath: Path, delimiter: str = ' ') -> tuple:
    """
    Read reference PES data from a text file.
    
    Parameters
    ----------
    filepath : Path
        Path to the reference data file
    delimiter : str
        Column separator
        
    Returns
    -------
    tuple
        (distances, energies) as numpy arrays
    """
    distances, energies = [], []
    
    with open(filepath) as f:
        lines = f.readlines()[1:]  # Skip header
        
        for line in lines:
            parts = [p for p in line.strip().split(delimiter) if p]
            if len(parts) >= 2:
                distances.append(float(parts[0]))
                energies.append(float(parts[1]))
    
    return np.array(distances), np.array(energies)


def bond_distance(geometries: np.ndarray) -> float:
    """Calculate the bond distance between the first two atoms."""
    return np.linalg.norm(geometries[1] - geometries[0])

---

## 5. Data Collection

In [16]:
def collect_pes_data(methods: list, base_dir: Path) -> dict:
    """
    Collect PES data for all methods from ORCA output files.
    
    Automatically discovers all .out files in each method subdirectory
    and processes them in alphabetical order.
    
    Parameters
    ----------
    methods : list
        List of method names to process
    base_dir : Path
        Base directory containing method subdirectories
        
    Returns
    -------
    dict
        Dictionary mapping method names to MethodResults objects
    """
    results = {m: MethodResults() for m in methods}
    
    for method in methods:
        method_dir = base_dir / method
        
        # Find all .out files and sort alphabetically
        out_files = sorted(method_dir.glob('*.out'))
        
        if not out_files:
            warnings.warn(f"No .out files found in {method_dir}")
            continue
        
        success_count = 0
        
        for path in out_files:
            try:
                data = read_orca_molecule(path)
                results[method].energies.append(data.energy)
                results[method].distances.append(bond_distance(data.geometries))
                results[method].scf_energies.append(data.dft_energies[1])  # def2-QZVPP
                results[method].t1_diagnostics.append(data.t1)
                results[method].s2_values.append(data.s2)
                success_count += 1
                
            except Exception as e:
                warnings.warn(f"Error parsing {path}: {e}")
        
        print(f"  {method}: {success_count}/{len(out_files)} points parsed")
    
    return results

In [17]:
def collect_atom_energies(atoms: list, methods: list) -> tuple:
    """
    Collect atomic reference energies for computing dissociation limits.
    
    Returns
    -------
    tuple
        (total_energies, scf_energies) lists for each method
    """
    energy_totals, scf_totals = [], []
    
    for method in methods:
        energy_sum, scf_sum = 0.0, 0.0
        file_method = METHOD_FILE_MAP[method]
        
        for atom in atoms:
            # Path format: species/Cr/Cr_ccsdt_hf_cbs_x2c.out
            filename = f'{atom}_{file_method}_cbs_x2c.out'
            path = SPECIES_DIR / atom / filename
            data = read_orca_atom(path)
            energy_sum += data.energy
            scf_sum += data.dft_energies[1] if len(data.dft_energies) > 1 else data.dft_energies[0]
        
        energy_totals.append(energy_sum)
        scf_totals.append(scf_sum)
    
    return energy_totals, scf_totals


def compute_binding_energies(results: dict, atom_energies: list, 
                              atom_scf_energies: list, methods: list) -> None:
    """
    Compute binding energies relative to atomic dissociation limits.
    Modifies results in-place.
    """
    for i, method in enumerate(methods):
        res = results[method]
        res.bde = (np.array(res.energies) - atom_energies[i]) * HARTREE_TO_KCAL
        res.scf_bde = (np.array(res.scf_energies) - atom_scf_energies[i]) * HARTREE_TO_KCAL

In [18]:
# Collect all PES data
print(f"Collecting PES data for {SPECIES}...")
print("-" * 40)
results = collect_pes_data(METHODS, BASE_DIR)

print("\nCollecting atomic reference energies...")
atom_energies, atom_scf_energies = collect_atom_energies(ATOMS, METHODS)

print("Computing binding energies...")
compute_binding_energies(results, atom_energies, atom_scf_energies, METHODS)

print("\n✓ Data collection complete!")

----------------------------------------
  ccsdt_hf: 84/84 points parsed
  ccsdt_svwn5: 119/119 points parsed
  ccsdt_pbe: 123/123 points parsed
  ccsdt_pw91: 122/122 points parsed
  ccsdt_r2scan: 122/122 points parsed
  ccsdt_pbe0: 123/123 points parsed

Computing binding energies...

✓ Data collection complete!


---

## 6. Load Reference Data

We compare our CC-DFT results against:

1. **Experimental PES** (Larsson 2022)
2. **Best theoretical estimate (BTE)** from high-level multireference calculations (Larsson 2022)

In [19]:
ref_data = {}

try:
    ref_data['exp'] = read_reference_data(BASE_DIR / 'exp.txt', delimiter=' ')
    print(f"✓ Loaded experimental data: {len(ref_data['exp'][0])} points")
except FileNotFoundError:
    print("⚠ Experimental data file not found: exp.txt")

try:
    ref_data['bte'] = read_reference_data(BASE_DIR / 'bte.txt', delimiter=' ')
    print(f"✓ Loaded BTE data: {len(ref_data['bte'][0])} points")
except FileNotFoundError:
    print("⚠ BTE data file not found: bte.txt")

✓ Loaded experimental data: 300 points
✓ Loaded BTE data: 1000 points


---

## 7. Export Diagnostic Data

In [20]:
def write_diagnostic_files(results: dict, methods: list, output_dir: Path) -> None:
    """Export diagnostic data to CSV files for each method."""
    output_dir.mkdir(exist_ok=True)
    
    for method in methods:
        res = results[method]
        filepath = output_dir / f'Cr2_{method}.csv'
        
        with open(filepath, 'w') as f:
            f.write('R,D_e,s^2,t1\n')
            for i in range(len(res.bde)):
                f.write(f'{res.distances[i]:.4f},{res.bde[i] / KCAL_TO_EV:.6f},'
                        f'{res.s2_values[i]:.4f},{res.t1_diagnostics[i]:.4f}\n')
        
        print(f"  Saved: {filepath}")


print("Exporting diagnostic data...")
write_diagnostic_files(results, METHODS, OUTPUT_DIR)
print("\n✓ Diagnostic files saved!")

Exporting diagnostic data...
  Saved: cr2_multi_diag/Cr2_ccsdt_hf.csv
  Saved: cr2_multi_diag/Cr2_ccsdt_svwn5.csv
  Saved: cr2_multi_diag/Cr2_ccsdt_pbe.csv
  Saved: cr2_multi_diag/Cr2_ccsdt_pw91.csv
  Saved: cr2_multi_diag/Cr2_ccsdt_r2scan.csv
  Saved: cr2_multi_diag/Cr2_ccsdt_pbe0.csv

✓ Diagnostic files saved!


---

## 8. Publication-Quality Figures

- **Interactive display** using Plotly (hover for R, E values)
- **Publication export** using Matplotlib 

In [21]:
def setup_plot_style():
    """Configure matplotlib font and font size"""
    plt.rcParams.update({
        'font.family': 'sans-serif',
        'font.sans-serif': ['DejaVu Sans', 'Helvetica', 'Arial'],
        'font.size': 8,
        'axes.titlesize': 9,
        'axes.labelsize': 8,
        'xtick.labelsize': 7,
        'ytick.labelsize': 7,
        'legend.fontsize': 7,
        'axes.linewidth': 0.8,
        'xtick.major.width': 0.6,
        'ytick.major.width': 0.6,
        'xtick.major.size': 3,
        'ytick.major.size': 3,
        'lines.linewidth': 1.3,
        'pdf.fonttype': 42,
        'ps.fonttype': 42,
    })


# Colorblind-safe palette
COLORS = {
    'ccsdt_hf':     '#56B4E9',  # Sky blue
    'ccsdt_svwn5':  '#CC79A7',  # Pink
    'ccsdt_pbe':    '#009E73',  # Bluish green
    'ccsdt_pw91':   '#E69F00',  # Orange
    'ccsdt_r2scan': '#0072B2',  # Blue
    'ccsdt_pbe0':   '#D55E00',  # Vermillion
}

LABELS_CC = {
    'ccsdt_hf':     'UCC-HF',
    'ccsdt_svwn5':  'UCC-SVWN5',
    'ccsdt_pbe':    'UCC-PBE',
    'ccsdt_pw91':   'UCC-PW91',
    'ccsdt_r2scan': r'UCC-R$^2$SCAN',
    'ccsdt_pbe0':   'UCC-PBE0',
}

LABELS_DFA = {
    'ccsdt_hf':     'UHF',
    'ccsdt_svwn5':  'USVWN5',
    'ccsdt_pbe':    'UPBE',
    'ccsdt_pw91':   'UPW91',
    'ccsdt_r2scan': r'UR$^2$SCAN',
    'ccsdt_pbe0':   'UPBE0',
}

setup_plot_style()
print("✓ Plot style configured")

✓ Plot style configured


### Figure 1: Main Manuscript Figure

In [22]:
def plot_main_figure(results: dict, methods: list, ref_data: dict):
    """
    Generate the main manuscript figure.
    - Matplotlib for publication-quality PDF/PNG export
    - Plotly for interactive notebook display
    """
    # === MATPLOTLIB: Publication Export ===
    fig_mpl, ax = plt.subplots(figsize=(4.5, 3.2))
    
    fail_methods = ['ccsdt_hf', 'ccsdt_pbe0', 'ccsdt_svwn5']
    work_methods = ['ccsdt_r2scan', 'ccsdt_pw91', 'ccsdt_pbe']
    
    for method in fail_methods:
        if method in methods and len(results[method].distances) > 0:
            res = results[method]
            ax.plot(res.distances, res.bde / KCAL_TO_EV,
                    color=COLORS[method], linestyle='-', linewidth=1.5,
                    label=LABELS_CC[method], zorder=2)
    
    for method in work_methods:
        if method in methods and len(results[method].distances) > 0:
            res = results[method]
            if method == 'ccsdt_pw91':
                ax.plot(res.distances, res.bde / KCAL_TO_EV,
                        color=COLORS[method], linestyle='--', linewidth=1.5,
                        label=LABELS_CC[method], zorder=4)
            else:
                ax.plot(res.distances, res.bde / KCAL_TO_EV,
                        color=COLORS[method], linestyle='-', linewidth=1.5,
                        label=LABELS_CC[method], zorder=3)
    
    if 'bte' in ref_data:
        ax.plot(ref_data['bte'][0], ref_data['bte'][1],
                color='#DC143C', linestyle='-', linewidth=1.5,
                label='BTE', zorder=5)
    if 'exp' in ref_data:
        ax.plot(ref_data['exp'][0], ref_data['exp'][1],
                color='black', linestyle='-', linewidth=1.5,
                label='Exp', zorder=6)
    
    ax.set_xlabel('Bond Length (Å)')
    ax.set_ylabel('Binding Energy (eV)')
    ax.set_xlim(1.40, 3.50)
    ax.set_ylim(-1.8, 0.3)
    ax.grid(True, which='major', linestyle='-', linewidth=0.5, alpha=0.4, zorder=0)
    ax.set_axisbelow(True)
    ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1.0),
              frameon=True, fancybox=False, edgecolor='lightgray',
              handlelength=1.8, labelspacing=0.4)
    
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(True)
        ax.spines[spine].set_linewidth(0.5)
    
    plt.tight_layout()
    fig_mpl.savefig(f'{SPECIES}_pes_main.pdf', dpi=300, bbox_inches='tight',
                    facecolor='white', transparent=False)
    fig_mpl.savefig(f'{SPECIES}_pes_main.png', dpi=300, bbox_inches='tight',
                    facecolor='white', transparent=False)
    plt.close(fig_mpl)
    print(f"✓ Saved: {SPECIES}_pes_main.pdf, {SPECIES}_pes_main.png")
    
    # === PLOTLY: Interactive Display ===
    fig = go.Figure()
    
    for method in fail_methods + work_methods:
        if method in methods and len(results[method].distances) > 0:
            res = results[method]
            if method == 'ccsdt_pw91':
                fig.add_trace(go.Scatter(
                    x=res.distances,
                    y=res.bde / KCAL_TO_EV,
                    mode='lines',
                    name=LABELS_CC[method],
                    line=dict(color=COLORS[method], width=2, dash='dash'),
                    hovertemplate='R = %{x:.3f} Å<br>E = %{y:.3f} eV<extra>' + LABELS_CC[method] + '</extra>'
                ))
            else:
                fig.add_trace(go.Scatter(
                    x=res.distances,
                    y=res.bde / KCAL_TO_EV,
                    mode='lines',
                    name=LABELS_CC[method],
                    line=dict(color=COLORS[method], width=2),
                    hovertemplate='R = %{x:.3f} Å<br>E = %{y:.3f} eV<extra>' + LABELS_CC[method] + '</extra>'
                ))
    
    if 'bte' in ref_data:
        fig.add_trace(go.Scatter(
            x=ref_data['bte'][0], y=ref_data['bte'][1],
            mode='lines', name='BTE',
            line=dict(color='#DC143C', width=2),
            hovertemplate='R = %{x:.3f} Å<br>E = %{y:.3f} eV<extra>BTE</extra>'
        ))
    if 'exp' in ref_data:
        fig.add_trace(go.Scatter(
            x=ref_data['exp'][0], y=ref_data['exp'][1],
            mode='lines', name='Exp',
            line=dict(color='black', width=2),
            hovertemplate='R = %{x:.3f} Å<br>E = %{y:.3f} eV<extra>Exp</extra>'
        ))
    
    fig.update_layout(
        title='Cr₂ Potential Energy Surface (CC-SCF Only)',
        xaxis_title='Bond Length (Å)',
        yaxis_title='Binding Energy (eV)',
        xaxis=dict(range=[1.40, 3.50]),
        yaxis=dict(range=[-1.8, 0.3]),
        template='plotly_white',
        font=dict(family='Arial', size=12),
        legend=dict(x=1.02, y=1, xanchor='left'),
        hovermode='closest',
        width=700, height=500
    )
    fig.show()


plot_main_figure(results, METHODS, ref_data)

✓ Saved: Cr-Cr_pes_main.pdf, Cr-Cr_pes_main.png


### Figure 2: Supporting Information Figure

In [23]:
def plot_si_figure(results: dict, methods: list, ref_data: dict):
    """
    Generate the SI figure with both CCSD(T) and HF/DFT curves.
    - Matplotlib for PDF/PNG export
    - Plotly for interactive notebook display
    """
    # === MATPLOTLIB: Publication Export ===
    fig_mpl, ax = plt.subplots(figsize=(5.5, 4))
    
    for method in methods:
        if len(results[method].distances) > 0:
            res = results[method]
            ax.plot(res.distances, res.bde / KCAL_TO_EV,
                    color=COLORS[method], linestyle='-', linewidth=1.5,
                    label=LABELS_CC[method])
            ax.plot(res.distances, res.scf_bde / KCAL_TO_EV,
                    color=COLORS[method], linestyle=':', linewidth=1.5,
                    label=LABELS_DFA[method])
    
    if 'bte' in ref_data:
        ax.plot(ref_data['bte'][0], ref_data['bte'][1],
                color='#DC143C', linestyle='-', linewidth=1.5, label='BTE')
    if 'exp' in ref_data:
        ax.plot(ref_data['exp'][0], ref_data['exp'][1],
                color='black', linestyle='-', linewidth=1.5, label='Exp')
    
    ax.set_xlabel('Bond Length (Å)')
    ax.set_ylabel('Binding Energy (eV)')
    ax.set_xlim(1.30, 3.50)
    ax.set_ylim(-3.5, 1.0)
    ax.grid(True, which='major', linestyle='-', linewidth=0.5, alpha=0.4, zorder=0)
    ax.set_axisbelow(True)
    ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1.0),
              frameon=True, fancybox=False, edgecolor='lightgray',
              fontsize=7, handlelength=1.8, labelspacing=0.3)
    
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(True)
        ax.spines[spine].set_linewidth(0.5)
    
    plt.tight_layout()
    fig_mpl.savefig(f'{SPECIES}_pes_SI.pdf', dpi=300, bbox_inches='tight',
                    facecolor='white', transparent=False)
    fig_mpl.savefig(f'{SPECIES}_pes_SI.png', dpi=300, bbox_inches='tight',
                    facecolor='white', transparent=False)
    plt.close(fig_mpl)
    print(f"✓ Saved: {SPECIES}_pes_SI.pdf, {SPECIES}_pes_SI.png")
    
    # === PLOTLY: Interactive Display ===
    fig = go.Figure()
    
    for method in methods:
        if len(results[method].distances) > 0:
            res = results[method]
            # CCSD(T)/CBS curve (solid)
            fig.add_trace(go.Scatter(
                x=res.distances, y=res.bde / KCAL_TO_EV,
                mode='lines', name=LABELS_CC[method],
                line=dict(color=COLORS[method], width=2),
                hovertemplate='R = %{x:.3f} Å<br>E = %{y:.3f} eV<extra>' + LABELS_CC[method] + '</extra>'
            ))
            # DFT/HF reference curve (dashed)
            fig.add_trace(go.Scatter(
                x=res.distances, y=res.scf_bde / KCAL_TO_EV,
                mode='lines', name=LABELS_DFA[method],
                line=dict(color=COLORS[method], width=2, dash='dot'),
                hovertemplate='R = %{x:.3f} Å<br>E = %{y:.3f} eV<extra>' + LABELS_DFA[method] + '</extra>'
            ))
    
    if 'bte' in ref_data:
        fig.add_trace(go.Scatter(
            x=ref_data['bte'][0], y=ref_data['bte'][1],
            mode='lines', name='BTE',
            line=dict(color='#DC143C', width=2),
            hovertemplate='R = %{x:.3f} Å<br>E = %{y:.3f} eV<extra>BTE</extra>'
        ))
    if 'exp' in ref_data:
        fig.add_trace(go.Scatter(
            x=ref_data['exp'][0], y=ref_data['exp'][1],
            mode='lines', name='Exp',
            line=dict(color='black', width=2),
            hovertemplate='R = %{x:.3f} Å<br>E = %{y:.3f} eV<extra>Exp</extra>'
        ))
    
    fig.update_layout(
        title='Cr₂ PES: CC-SCF vs SCF',
        xaxis_title='Bond Length (Å)',
        yaxis_title='Binding Energy (eV)',
        xaxis=dict(range=[1.30, 3.50]),
        yaxis=dict(range=[-3.5, 1.0]),
        template='plotly_white',
        font=dict(family='Arial', size=12),
        legend=dict(x=1.02, y=1, xanchor='left'),
        hovermode='closest',
        width=800, height=550
    )
    fig.show()


plot_si_figure(results, METHODS, ref_data)

✓ Saved: Cr-Cr_pes_SI.pdf, Cr-Cr_pes_SI.png


---

## 9. Summary Statistics

Equilibrium bond lengths ($R_e$) and dissociation energies ($D_e$) at the potential energy minimum.

Values in parentheses show deviation from experiment (positive = overestimate, negative = underestimate).

In [24]:
# Get experimental reference values
if 'exp' in ref_data:
    exp_min_idx = np.argmin(ref_data['exp'][1])
    exp_r_e = ref_data['exp'][0][exp_min_idx]
    exp_d_e = -ref_data['exp'][1][exp_min_idx]
else:
    exp_r_e, exp_d_e = None, None

def fmt_with_delta(val, ref):
    """Format value with delta in parentheses."""
    if ref is None:
        return f"{val:.3f}"
    delta = val - ref
    return f"{val:.3f} ({delta:+.3f})"

# === HF and KS-DFT Reference Results ===
print("HF and KS-DFT")
print("=" * 55)
print(f"{'Method':<15} {'R_e (Å)':<20} {'D_e (eV)':<20}")
print("-" * 55)

for method in METHODS:
    res = results[method]
    if len(res.distances) > 0:
        scf_bde_ev = res.scf_bde / KCAL_TO_EV
        min_idx = np.argmin(scf_bde_ev)
        r_e = res.distances[min_idx]
        d_e = -scf_bde_ev[min_idx]
        print(f"{LABELS_DFA[method]:<15} {fmt_with_delta(r_e, exp_r_e):<20} {fmt_with_delta(d_e, exp_d_e):<20}")

print()

# === CCSD(T) Results ===
print("CCSD(T)/CBS")
print("=" * 85)
print(f"{'Method':<15} {'R_e (Å)':<20} {'D_e (eV)':<20} {'T1/QZ':<8} {'⟨S²⟩/QZ':<8}")
print("-" * 85)

for method in METHODS:
    res = results[method]
    if len(res.distances) > 0:
        bde_ev = res.bde / KCAL_TO_EV
        min_idx = np.argmin(bde_ev)
        r_e = res.distances[min_idx]
        d_e = -bde_ev[min_idx]
        t1_eq = res.t1_diagnostics[min_idx]
        s2_eq = res.s2_values[min_idx]
        print(f"{LABELS_CC[method]:<15} {fmt_with_delta(r_e, exp_r_e):<20} {fmt_with_delta(d_e, exp_d_e):<20} {t1_eq:<10.4f} {s2_eq:<10.3f}")

print()

# === Multireference Wavefunction Method ===
print("Multireference Wavefunction Method")
print("=" * 55)
print(f"{'Method':<15} {'R_e (Å)':<20} {'D_e (eV)':<20}")
print("-" * 55)

if 'bte' in ref_data:
    bte_min_idx = np.argmin(ref_data['bte'][1])
    bte_r_e = ref_data['bte'][0][bte_min_idx]
    bte_d_e = -ref_data['bte'][1][bte_min_idx]
    print(f"{'BTE':<15} {fmt_with_delta(bte_r_e, exp_r_e):<20} {fmt_with_delta(bte_d_e, exp_d_e):<20}")

print()

# === Experiment ===
print("Experiment")
print("=" * 40)
print(f"{'Source':<20} {'R_e (Å)':<10} {'D_e (eV)':<10}")
print("-" * 40)

if 'exp' in ref_data:
    print(f"{'Larsson 2022':<20} {exp_r_e:<10.3f} {exp_d_e:<10.3f}")

HF and KS-DFT
Method          R_e (Å)              D_e (eV)            
-------------------------------------------------------
UHF             3.380 (+1.703)       0.071 (-1.499)      
USVWN5          1.580 (-0.097)       3.262 (+1.692)      
UPBE            1.700 (+0.023)       1.679 (+0.109)      
UPW91           1.700 (+0.023)       1.735 (+0.165)      
UR$^2$SCAN      2.480 (+0.803)       0.854 (-0.716)      
UPBE0           2.580 (+0.903)       0.851 (-0.719)      

CCSD(T)/CBS
Method          R_e (Å)              D_e (eV)             T1/QZ    ⟨S²⟩/QZ 
-------------------------------------------------------------------------------------
UCC-HF          2.380 (+0.703)       1.053 (-0.517)       0.0388     5.494     
UCC-SVWN5       1.620 (-0.057)       1.729 (+0.159)       0.0287     0.712     
UCC-PBE         1.640 (-0.037)       1.569 (-0.001)       0.0234     1.291     
UCC-PW91        1.640 (-0.037)       1.569 (-0.001)       0.0235     1.329     
UCC-R$^2$SCAN   1.680 (+0.003

---

## References

### Experimental and Theoretical Reference Data
- Larsson, H. R.; Zhai, H.; Umrigar, C. J.; Chan, G. K.-L. The Chromium Dimer: Closing a Chapter of Quantum Chemistry. *J. Am. Chem. Soc.* **2022**, *144*, 15932–15937. [DOI: 10.1021/jacs.2c06357](https://doi.org/10.1021/jacs.2c06357)
- Data for BTE and Exp were obtained from [Larsson's GitHub page](https://github.com/h-larsson/Cr2Pes22)

### Computational Software
- Neese, F. et al. Software Update: The ORCA Program System—Version 6.0. *WIREs Comput. Mol. Sci.* **2025**, e70019. [DOI: 10.1002/wcms.70019](https://doi.org/10.1002/wcms.70019)

---

## License

This code is released under the MIT License.